# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [3]:
# import libraries

# libraries for data manipulation & loading
import pandas as pd
import numpy as np
from sqlalchemy import create_engine
import pickle

# libraries for NLP
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.tokenize import word_tokenize

nltk.download(['punkt', 'stopwords', 'wordnet', 'averaged_perceptron_tagger'])

# libraries for ML
from sklearn.multioutput import MultiOutputClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer, TfidfVectorizer
from sklearn.metrics import classification_report
from sklearn.naive_bayes import MultinomialNB


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


In [4]:
# load data from database
engine = create_engine('sqlite:///etl_done.db')
df = pd.read_sql_table('fact_messages',engine)
X = df['message'].values
Y = df.iloc[:,4:].values
categories = df.iloc[:,4:].columns
print(X.shape, Y.shape)

(26215,) (26215, 36)


### 2. Write a tokenization function to process your text data

In [154]:
def tokenize(text):
    # instantiate the stopwords list & lemmatizer
    stop_words = stopwords.words("english")
    lemmatizer = WordNetLemmatizer()
    
    # normalize case and remove punctuation
    text = re.sub(r"[^a-zA-Z0-9]", " ", text.lower())
    
    # tokenize text
    tokens = word_tokenize(text)
    
    # lemmatize andremove stop words
    tokens = [lemmatizer.lemmatize(word) for word in tokens if word not in stop_words]

    return tokens

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [195]:
pipeline = Pipeline([
        ('vect', CountVectorizer(tokenizer=tokenize)),
        ('tfidf', TfidfTransformer()),
        ('clf', MultiOutputClassifier(RandomForestClassifier()))
])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [196]:
X_train, X_test, y_train, y_test = train_test_split(X, Y)

pipeline.fit(X_train, y_train)

Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...oob_score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=1))])

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [158]:
y_pred = pipeline.predict(X_test)

for i, element in enumerate(np.transpose(y_pred)):
    print(categories[i])
    print(classification_report(np.transpose(y_pred)[i], np.transpose(y_test)[i]))

             precision    recall  f1-score   support

          0       0.47      0.63      0.54      1133
          1       0.92      0.85      0.88      5421

avg / total       0.84      0.81      0.82      6554

             precision    recall  f1-score   support

          0       0.98      0.89      0.93      5931
          1       0.44      0.83      0.57       623

avg / total       0.93      0.88      0.90      6554

             precision    recall  f1-score   support

          0       1.00      0.99      1.00      6554
          1       0.00      0.00      0.00         0

avg / total       1.00      0.99      1.00      6554

             precision    recall  f1-score   support

          0       0.86      0.75      0.80      4303
          1       0.62      0.76      0.68      2251

avg / total       0.77      0.75      0.76      6554

             precision    recall  f1-score   support

          0       1.00      0.92      0.96      6482
          1       0.09      0.67 

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1137: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


### 6. Improve your model
Use grid search to find better parameters. 

In [198]:
parameters = {'clf__estimator__criterion': ["gini", "entropy"],     
        'clf__estimator__n_jobs':[-1]
        }

cv = GridSearchCV(pipeline, parameters, n_jobs=-1)

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [199]:
cv.fit(X_train, y_train)

y_pred = cv.predict(X_test)

for i, element in enumerate(np.transpose(y_pred)):
    print(categories[i])
    print(classification_report(np.transpose(y_pred)[i], np.transpose(y_test)[i]))

related
             precision    recall  f1-score   support

          0       0.44      0.66      0.53      1026
          1       0.93      0.84      0.88      5528

avg / total       0.85      0.81      0.83      6554

request
             precision    recall  f1-score   support

          0       0.98      0.90      0.94      5966
          1       0.43      0.80      0.56       588

avg / total       0.93      0.89      0.90      6554

offer
             precision    recall  f1-score   support

          0       1.00      1.00      1.00      6554
          1       0.00      0.00      0.00         0

avg / total       1.00      1.00      1.00      6554

aid_related
             precision    recall  f1-score   support

          0       0.86      0.76      0.80      4369
          1       0.61      0.75      0.67      2185

avg / total       0.77      0.75      0.76      6554

medical_help
             precision    recall  f1-score   support

          0       1.00      0.92      0

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1137: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [202]:
pipeline_ = Pipeline([
        #('vect', CountVectorizer(tokenizer=tokenize)),
        #('tfidf', TfidfTransformer()),
        ('tfidf', TfidfVectorizer(tokenizer=tokenize)),
        ('clf', MultiOutputClassifier(RandomForestClassifier()))
        #('clf', MultiOutputClassifier(MultinomialNB()))
])

cv_ = GridSearchCV(pipeline_, parameters, n_jobs=-1)

cv_.fit(X_train, y_train)

y_pred = cv_.predict(X_test)

for i, element in enumerate(np.transpose(y_pred)):
    print(categories[i])
    print(classification_report(np.transpose(y_pred)[i], np.transpose(y_test)[i]))

related
             precision    recall  f1-score   support

          0       0.46      0.61      0.52      1153
          1       0.91      0.85      0.88      5401

avg / total       0.83      0.80      0.82      6554

request
             precision    recall  f1-score   support

          0       0.98      0.90      0.93      5947
          1       0.44      0.79      0.56       607

avg / total       0.93      0.89      0.90      6554

offer
             precision    recall  f1-score   support

          0       1.00      1.00      1.00      6554
          1       0.00      0.00      0.00         0

avg / total       1.00      1.00      1.00      6554

aid_related
             precision    recall  f1-score   support

          0       0.86      0.75      0.80      4409
          1       0.60      0.76      0.67      2145

avg / total       0.78      0.75      0.76      6554

medical_help
             precision    recall  f1-score   support

          0       1.00      0.93      0

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1137: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


### 9. Export your model as a pickle file

In [205]:
filename = 'classifier.pkl'
pickle.dump(cv_, open(filename, 'wb'))

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.